In [49]:
import pandas as pd 
import numpy as np 
import re
import stanza
nlp = stanza.Pipeline('ta')

c:\Users\Vasantha Raj\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-25 17:20:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-12-25 17:20:39 INFO: Downloaded file to C:\Users\Vasantha Raj\stanza_resources\resources.json
2024-12-25 17:20:41 INFO: Loading these models for language: ta (Tamil):
| Processor | Package      |
----------------------------
| tokenize  | ttb          |
| mwt       | ttb          |
| pos       | ttb_nocharlm |
| lemma     | ttb_nocharlm |
| depparse  | ttb_nocharlm |

2024-12-25 17:20:41 INFO: Using device: cpu
2024-12-25 17:20:41 INFO: Loading: tokenize
c:\Users\Vasantha Raj\AppData\Local\P

In [53]:
data=pd.read_csv(r"../data\PS_train.csv")
data1=pd.read_csv(r'../data\PS_train.csv')
data.head()

,content,labels
0,தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...,Neutral
1,அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...,Substantiated
2,ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து......,Opinionated
3,"எடப்பாடியை கண்டுகொள்ளாத ""எடப்பாடி""🫢\n ---\nஆதர...",Positive
4,எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது \n#மக...,Opinionated


In [54]:

def extract_components(input_text):
    # Regex patterns
    tamil_regex = r"[\u0B80-\u0BFF\s]+"  # Tamil characters
    emoji_regex = (
        r"[\U0001F1E6-\U0001F1FF]|[\U0001F300-\U0001F5FF]|[\U0001F600-\U0001F64F]|"
        r"[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|"
        r"[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|"
        r"[\U0001FA70-\U0001FAFF]|[\U00002702-\U000027B0]|[\U000024C2-\U0001F251]|"
        r"[\U0001F004]|[\U0001F0CF]|[\u2600-\u26FF]|[\u2700-\u27BF]|[\uFE0F]|[\u2300-\u23FF]"
    )
    hashtag_regex = r"#\S+"  # Hashtags
    
    # Results
    tamil_sentences = []
    emojis = []
    tags = []
    
    # Split input into lines
    lines = [line.strip() for line in input_text.splitlines() if line.strip()]
    
    for line in lines:
        # Extract Tamil sentences
        tamil_matches = re.findall(tamil_regex, line)
        if tamil_matches:
            tamil_sentences.extend([match.strip() for match in tamil_matches if match.strip()])
        
        # Extract emojis
        emoji_matches = re.findall(emoji_regex, line)
        if emoji_matches:
            emojis.extend(emoji_matches)
        
        # Extract hashtags
        hashtag_matches = re.findall(hashtag_regex, line)
        if hashtag_matches:
            tags.extend(hashtag_matches)
    query=" ".join(tamil_sentences)
    doc = nlp(query)
    tokens=[]
    lemma=[]
    pos=[]
    dependancy_relation=[]
    for sentence in doc.sentences:
        for word in sentence.words:
            tokens.append(word.text)
            lemma.append(word.lemma)
            pos.append(word.upos)
            dependancy_relation.append(word.deprel)
    return {
        "tamil_sentences": [query],
        "emojis": emojis,
        "tags": tags,
        "tokens":tokens,
        "lemmas":lemma,
        "pos":pos,
        "dependancy_relation":dependancy_relation
    }


In [55]:
data['content']=data['content'].apply(extract_components)

dev and train both are same

raw data

In [57]:
for i in range(5):
    print(data1['content'][i],data1['labels'][i])
    print('-----------------------------------------------')

தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் டாக்டர். கிருஷ்ணசாமியை  ஆதரித்து தேமுதிக பொதுச் செயலாளர் பிரேமலதா விஜயகாந்த்  ஸ்ரீவில்லிபுத்தூரில் பிரச்சாரம்.
#News18TamilNadu #LokSabaElection2024  Neutral
-----------------------------------------------
அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டியில் சொல்லிட்டார்..!

           Substantiated
-----------------------------------------------
ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து... இன்றுவரை ஒரு ஆணியை கூட பிடுங்காத தமிழக அரசு

#DMKFailsTN



 Opinionated
-----------------------------------------------
எடப்பாடியை கண்டுகொள்ளாத "எடப்பாடி"🫢
 ---
ஆதரிப்பீர் 🌄 உதயசூரியன் 🏴🚩
---

#மக்களின்குரல் #ஓமலூர் #சேலம் #சேலம்4செல்வகணபதி #Vote4DMK #Vote4INDIA   Stalin #Testimonials QA #சேலம்தெற்கு #சேலம்வடக்கு #எடப்பாடி #சேலம்மேற்கு #Eps #Edappadi #Palanisamy  Positive
-----------------------------------------------
எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது 
#மக்களின்_சின்னம்_மைக்  Opinionated
-----------------------------------------------


processed data

In [58]:
for i in range(5):
    print(data['content'][i],data['labels'][i])
    print('-----------------------------------------------')

{'tamil_sentences': ['தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் டாக்டர் கிருஷ்ணசாமியை  ஆதரித்து தேமுதிக பொதுச் செயலாளர் பிரேமலதா விஜயகாந்த்  ஸ்ரீவில்லிபுத்தூரில் பிரச்சாரம்'], 'emojis': [], 'tags': ['#News18TamilNadu', '#LokSabaElection2024'], 'tokens': ['தென்காசி', 'தொகுதி', 'புதிய', 'தமிழகம்', 'கட்சி', 'வேட்பாளர்', 'டாக்டர்', 'கிருஷ்ணசாமியை', 'ஆதரித்து', 'தேமுதிக', 'பொதுச்', 'செயலாளர்', 'பிரேமலதா', 'விஜயகாந்த்', 'ஸ்ரீவில்லிபுத்தூரில்', 'பிரச்சாரம்'], 'lemmas': ['தென்காசி', 'தொகுதி', 'புதிய', 'தமிழகம்', 'கட்சி', 'வேட்பாளர்', 'டாக்டர்', 'கிருஷ்ணசாமி', 'ஆதரி', 'தேமுதிக', 'பொது', 'செயலாளர்', 'பிரேமலதா', 'விஜயகாந்த்', 'ஸ்ரீவில்லிபுத்தூ்', 'பிரச்சாரம்'], 'pos': ['PROPN', 'NOUN', 'ADJ', 'PROPN', 'NOUN', 'NOUN', 'PROPN', 'PROPN', 'VERB', 'PROPN', 'NOUN', 'NOUN', 'PROPN', 'PROPN', 'PROPN', 'NOUN'], 'dependancy_relation': ['nmod', 'nmod', 'amod', 'nmod', 'nmod', 'nsubj', 'nmod', 'obj', 'amod', 'nmod', 'nmod', 'nmod', 'nmod', 'nsubj', 'obl', 'root']} Neutral
--------------------------------

In [59]:
data['labels'].unique()

array(['Neutral', 'Substantiated', 'Opinionated', 'Positive', 'Sarcastic',
       'Negative', 'None of the above'], dtype=object)

In [60]:
d={'Neutral':0, 'Substantiated':1, 'Opinionated':2, 'Positive':5, 'Sarcastic':4,
       'Negative':-5, 'None of the above':-1}

In [61]:
data['labels']=data['labels'].apply(lambda x: d[x])

In [62]:
data.to_csv("preprocessed_data.csv")

In [26]:
# pip install stanza

In [14]:
# import stanza
# stanza.download('ta')

c:\Users\Vasantha Raj\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-25 15:22:48 INFO: Downloaded file to C:\Users\Vasantha Raj\stanza_resources\resources.json
2024-12-25 15:22:48 INFO: Downloading default packages for language: ta (Tamil) ...
2024-12-25 15:33:06 INFO: Downloaded file to C:\Users\Vasantha Raj\stanza_resources\ta\default.zip
2024-12-25 15:33:09 INFO: Finished downloading models and saved to C:\Users\Vasantha Raj\stanza_resources
